In [28]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import os
import time

In [29]:
def read_top_instance(file_path):
    with open(file_path, 'r') as file:
        # Read the first three lines
        n_line = file.readline().strip()
        m_line = file.readline().strip()
        tmax_line = file.readline().strip()

        # Parse n N
        _, N = n_line.split()
        N = int(N)

        # Parse m P
        _, P = m_line.split()
        P = int(P)

        # Parse tmax Tmax
        _, Tmax = tmax_line.split()
        Tmax = float(Tmax)

        # Read the remaining lines for each node
        nodes = []
        for i in range(N):
            line = file.readline().strip()
            if line == '':
                continue  # Skip empty lines
            x_str, y_str, s_str = line.split()
            x = float(x_str)
            y = float(y_str)
            s = float(s_str)
            nodes.append({'id': i, 'x': x, 'y': y, 'score': s})
        
        return N, P, Tmax, nodes


In [30]:

def compute_distance_matrix(nodes):
    N = len(nodes)
    distance_matrix = np.zeros((N, N))
    for i in range(N):
        xi, yi = nodes[i]['x'], nodes[i]['y']
        for j in range(N):
            xj, yj = nodes[j]['x'], nodes[j]['y']
            distance = np.hypot(xi - xj, yi - yj)
            distance_matrix[i][j] = distance
    return distance_matrix


In [31]:
def solve_top_gurobi(N, P, Tmax, nodes, distance_matrix):
    try:
        

        # Start timing
        start_time = time.time()
    
        # Create a new model
        model = gp.Model('TOP_base')
        model.Params.TimeLimit = 600
        model.Params.MIPGap = 0.01 
        model.Params.OutputFlag = 0

        # Sets
        V = range(N)  # Set of nodes
        K = range(P)  # Set of vehicles

        # Parameters
        c = distance_matrix
        p = [node['score'] for node in nodes]

        # Decision variables
        x = model.addVars(V, V, K, vtype=GRB.BINARY, name='x')
        y = model.addVars(V, K, vtype=GRB.BINARY, name='y')

        # Objective function
        model.setObjective(
            gp.quicksum(p[j] * y[j, k] for j in V for k in K),
            GRB.MAXIMIZE
        )

        # Constraints
        for k in K:
            # Time budget constraint
            model.addConstr(
                gp.quicksum(c[i][j] * x[i, j, k] for i in V for j in V if i != j) <= Tmax,
                name=f'TimeBudget_{k}'
            )

            # Flow conservation
            for j in V:
                if j != 0 and j != N - 1:
                    model.addConstr(
                        gp.quicksum(x[i, j, k] for i in V if i != j) == y[j, k],
                        name=f'FlowIn_{j}_{k}'
                    )
                    model.addConstr(
                        gp.quicksum(x[j, i, k] for i in V if i != j) == y[j, k],
                        name=f'FlowOut_{j}_{k}'
                    )

            # Start at start node
            model.addConstr(
                gp.quicksum(x[0, j, k] for j in V if j != 0) == 1,
                name=f'StartAtStartNode_{k}'
            )

            # End at end node
            model.addConstr(
                gp.quicksum(x[j, N - 1, k] for j in V if j != N - 1) == 1,
                name=f'EndAtEndNode_{k}'
            )

            # No flow into start node and no flow out of end node
            model.addConstr(
                gp.quicksum(x[j, 0, k] for j in V if j != 0) == 0,
                name=f'NoFlowIntoStartNode_{k}'
            )
            model.addConstr(
                gp.quicksum(x[N - 1, j, k] for j in V if j != N - 1) == 0,
                name=f'NoFlowOutOfEndNode_{k}'
            )

        # Each node is visited at most once
        for j in V:
            if j != 0 and j != N - 1:
                model.addConstr(
                    gp.quicksum(y[j, k] for k in K) <= 1,
                    name=f'VisitOnce_{j}'
                )

        # Find the maximum distance
        max_distance = np.max(distance_matrix)

        # Set the big M value
        M = Tmax + max_distance # Or consider using M = Tmax + max_distance if appropriate

        # Subtour elimination constraints (MTZ)
        u = model.addVars(V, K, vtype=GRB.CONTINUOUS, lb=0, ub=Tmax, name='u')

        for k in K:
            for i in V:
                for j in V:
                    if i != j :
                        model.addConstr(
                            u[i, k] + c[i][j] - M * (1 - x[i, j, k]) <= u[j, k],
                            name=f'Subtour_{i}_{j}_{k}'
                        )

        # Optimize the model
        model.optimize()
        end_time = time.time()
        run_time = end_time - start_time

        # Initialize the routes dictionary and collected prize
        routes = {}
        total_collected_prize = 0


        # Initialize the routes dictionary
        routes = {}

        # Extract the solution
        if model.Status == GRB.OPTIMAL or model.Status == GRB.TIME_LIMIT:
            for k in K:
                # Get the arcs used by vehicle k
                arcs = [(i, j) for i in V for j in V if x[i, j, k].X > 0.5]
                if not arcs:
                    continue
                # Store the arcs in the routes dictionary
                routes[k] = arcs

                # Reconstruct the route to calculate the collected prize
                arc_dict = {i: j for (i, j) in arcs}
                route = [0]
                current_node = 0
                while current_node != N - 1:
                    if current_node in arc_dict:
                        next_node = arc_dict[current_node]
                        route.append(next_node)
                        current_node = next_node
                    else:
                        break
                # Calculate collected prize for this vehicle
                collected_prize = sum(p[j] for j in route if j != 0 and j != N - 1)
                total_collected_prize += collected_prize
        elif model.Status == GRB.INFEASIBLE:
            print("Model is infeasible.")
            return None, None, None, None
        else:
            print(f"Optimization was stopped with status {model.Status}")
            return None, None, None, None

        # Get the optimality gap
        if model.Status == GRB.OPTIMAL:
            gap = 0.0
        else:
            gap = model.MIPGap

        return routes, run_time, gap, total_collected_prize

    except gp.GurobiError as e:
        print(f"Gurobi Error: {e}")
        return None, None, None, None
    except AttributeError as e:
        print(f"Attribute error: {e}")
        return None, None, None, None

In [32]:
def process_all_instances(directory_path):
    import csv
    import os

    # Prepare a list to store the results
    results = []

    csv_file = 'results.csv'
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Instance', 'RunTime', 'Gap', 'TotalCollectedPrize', 'AveragePrizePerVehicle'])

    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):  # Adjust the extension if needed
            file_path = os.path.join(directory_path, filename)
            print(f"\nProcessing instance: {filename}")
            N, P, Tmax, nodes = read_top_instance(file_path)
            distance_matrix = compute_distance_matrix(nodes)
            routes, run_time, gap, total_collected_prize = solve_top_gurobi(N, P, Tmax, nodes, distance_matrix)

            if routes is not None:
                # Calculate average prize per vehicle
                num_vehicles_used = len(routes)
                if num_vehicles_used > 0:
                    average_prize = total_collected_prize / num_vehicles_used
                else:
                    average_prize = 0

                # Append the results
                results.append({
                    'Instance': filename,
                    'RunTime': run_time,
                    'Gap': gap,
                    'TotalCollectedPrize': total_collected_prize,
                    'AveragePrizePerVehicle': average_prize
                })

                # Write the results to the CSV file
                with open(csv_file, mode='a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([filename, run_time, gap, total_collected_prize, average_prize])
            else:
                print(f"No solution found for instance {filename}.")

    return results

In [33]:
#test with first instance
if __name__ == "__main__":
    # Path to a single instance file
    instance_file = 'Set_100_234/p4.2.a.txt'  # Replace with actual file name

    # Read the instance data
    N, P, Tmax, nodes = read_top_instance(instance_file)

    # Compute the distance matrix
    distance_matrix = compute_distance_matrix(nodes)

    # Solve the TOP using Gurobi
    results  = solve_top_gurobi(N, P, Tmax, nodes, distance_matrix)
    print(results)

Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
({0: [(0, 96), (7, 34), (23, 7), (34, 76), (76, 99), (96, 23)], 1: [(0, 14), (14, 52), (24, 99), (52, 55), (55, 78), (78, 24)]}, 13.382877588272095, 0.0, 206.0)


In [34]:

if __name__ == "__main__":
    # Set the path to your instance directory
    instance_directory = 'Set_100_234'  

    # Process all instances in the directory and collect results
    results = process_all_instances(instance_directory)

    print("\nSummary of Results:")
    for result in results:
        print(f"Instance: {result['Instance']}, Run Time: {result['RunTime']:.2f}s, "
              f"Gap: {result['Gap']*100:.2f}%, Total Prize: {result['TotalCollectedPrize']}, "
              f"Avg Prize/Vehicle: {result['AveragePrizePerVehicle']}")


Processing instance: p4.2.a.txt
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01



Processing instance: p4.2.b.txt
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01

Processing instance: p4.2.c.txt
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01

Processing instance: p4.2.d.txt
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01

Processing instance: p4.2.e.txt
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01

Processing instance: p4.2.f.txt
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01

Processing instance: p4.2.g.txt
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01

Processing instance: p4.2.h.txt
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01

Processing instance: p4.2.i.txt
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01

Processing instance: p4.2.j.txt
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01

Processing instance: p4.2.k.txt
Set parameter TimeLimi